## Load house prices

In [ ]:
text_RDD = sc.textFile("file:///oasis/scratch/comet/zonca/temp_project/houses_1mil.txt", minPartitions=48)

In [ ]:
def mapper_parse_lines(line):
    """Parse line into (neighborhoood, price) pair"""
    words = line.split()
    return (words[1], float(words[2]), int(words[0]))

In [ ]:
house_prices_RDD = text_RDD.map(mapper_parse_lines)

In [ ]:
house_prices_RDD.take(5)

In [ ]:
house_prices_df = sqlContext.createDataFrame(house_prices_RDD, ["neighborhood", "price", "bedrooms"])

In [ ]:
house_prices_df

In [ ]:
house_prices_df.printSchema()

# Prepare features

In [ ]:
from pyspark.ml.feature import StringIndexer

stringIndexer = StringIndexer(inputCol="neighborhood",
                              outputCol="neighborhood_index")


In [ ]:
house_prices_df_indexed = stringIndexer.fit(house_prices_df).transform(house_prices_df)

In [ ]:
house_prices_df_indexed.show()

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [ ]:
def create_features(row):
    return Row(label=row.price,
               features=Vectors.dense([row.bedrooms, row.neighborhood_index])
              )
    
house_prices_features = sqlContext.createDataFrame(
    house_prices_df_indexed.rdd.map(create_features))

In [ ]:
house_prices_features.show()

# Fit a Random Forest Regressor

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
RandomForestRegressor?

In [ ]:
regr = RandomForestRegressor(numTrees=10, featureSubsetStrategy="all")
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + regr.explainParams())

In [ ]:
# Learn a RandomForestRegressor model. This uses the parameters stored in regr.
model = regr.fit(house_prices_features)

In [ ]:
output = model.transform(house_prices_features)

In [ ]:
output.show()

# Compute mean squared error

In [ ]:
from pyspark.sql.functions import pow

In [ ]:
mean_squared_error = output.withColumn(
    "squared_error",
    pow(output.label - output.prediction,2)
    ).agg({"squared_error":"mean"}).collect()

In [ ]:
mean_squared_error

In [ ]:
import numpy as np
np.sqrt(mean_squared_error[0][0])